# 2.2 Ітераційні методи розв'язування СЛАР
---------------------

## 2.2.4.  Метод Зейделя

Нехай маємо СЛАР

$(1)\qquad\qquad\qquad A\,x = b,$

де  $x \in \mathbb{R}^{n}$  --- невідомий вектор, $n \ge 2$ - довільне натуральне число,
$A\in \mathbb{R}^{n\times n},\; b\in \mathbb{R}^{n}$ - задані квадратна матриця і вектор вільних членів відповідно:

$(2)\qquad\qquad
A=
\begin{pmatrix}
a_{11} & a_{12} &\ldots & a_{1n} \\
a_{21} & a_{22} &\ldots & a_{2n} \\
\ldots & \ldots & \ldots \\
a_{n1} & a_{n2} &\ldots & a_{nn}
\end{pmatrix},\quad  
b=
\begin{pmatrix}
b_{1}\\
b_{2}\\
\ldots  \\
b_{n}
\end{pmatrix}.
$

Вважаємо, що $\mathrm{det} A\neq 0$, а також $a_{ii} \neq 0, \; i \in \overline {1, n}$.


Метод *Зейделя* полягає в обчисленні послідовності 

$\qquad\qquad x^0, x^1, \ldots, x^k, \ldots$

наближень розв'язку системи за правилом:

$(3)\qquad\qquad\qquad
x_{i}^{k+1} =  - \sum_{j=1}^{i-1} \frac{a_{ij}}{a_{ii}} x_j^{k+1} - \sum_{j=i+1}^n
\frac{a_{ij}}{a_{ii}} x_j^k + d_i,\quad
 i =\overline{1,n},\ \ \ k\in \mathbb{N}_0.
$

де 
$x^k =(x_1^k,x_2^k, \dots , x_n^k)^\top$ -- $k$-та ітерація вектора наближень, $x^0 \in \mathbb{R}^n$ -- початкове наближення.


#### Пояснення до використання програмного коду
-----------------

*   Підготувати обчислювальне середовище і потрібні функції :
    1. виконати комірку для підготовки середовища
    2. виконати комірку, де визначені допоміжні функції  ``norm_1``,``norm_2`` i ``norm_3``    
    3. виконати комірку, де визначена функція ``simple_iteration`` 
    4. виконати комірку, де визначені функції ``Jacobi_modification`` і ``Seidel_solver``
*   Обчислити шуканий розв'язок заданої СЛАР :
    1. виконати комірки, де визначені функції ``set_matrix`` i ``set_vector`` 
    2. виконати комірку, де визначена функція ``set_x0`` 
    3. виконати комірку із заданням точності ``eps`` і максимальної кількості ітерацій ``max_iter``
    4. Виконати комірку з викликом функції``Seidel_solver``

#### Програмна реалізація методу
------------

>#### Підготовка середовища

In [1]:
import numpy as np

>#### Допоміжні функції

In [2]:
def norm_1(a):
    """обчислення норми_1 матриці a"""   
    m=0
    for i in range(a.shape[0]):
        s=0
        for j in range(a.shape[1]):
            s+=abs(a[i][j])
        if s>m:
            m=s
    return m

def norm_2(a):
    """обчислення норми_2 матриці a"""   
    m=0
    for j in range(a.shape[1]):
        s=0
        for i in range(a.shape[0]):
            s+=abs(a[i][j])
        if s>m:
            m=s
    return m

def norm_3(a):
    """обчислення евклідової норми вектора a"""
    return np.sqrt(np.sum(a**2))

>#### ``Jacobi_modification`` -- функція, яка модифікує матрицю і вектор вільних членів СЛАР (1)     

-------------------

In [3]:
def Jacobi_modification(a, b):
    """ модифікація матриці і вектора вільних членів СЛАР """
    for i in range(a.shape[0]):
        b[i] /= a[i,i]
        a[i,:] /= -a[i,i]
        a[i,i]=0
        

>#### ``Seidel_iteration`` -- функція, яка реалізує метод Зейделя для розв'язування СЛАР

-------------------

In [4]:
def Seidel_iteration(b,d,x0,eps,iter_print):
    """ послідовно обчислює наближення розв'язку СЛАР 
      за методом Зейделя, поки евклідова норма 
      різниці двох послідовних наближень не стане меншою заданої точності eps      
      """
    n=b.shape[0]
    x_prev=x0.copy()
    
    k=1
    x_n = np.empty(n) 
    x_n[0]=np.matmul(b[0,1:],x_prev[1:]).sum() +d[0]
    for i in range(1,n):
        x_n[i]= np.matmul(b[i,:i],x_n[:i]).sum() + np.matmul(b[i,i+1:],x_prev[i+1:]).sum() +d[i] 

    while norm_3(x_n - x_prev)  > eps and k < max_iter:        
        x_prev=x_n.copy()  
        
        k+=1
        x_n[0]=np.matmul(b[0,1:],x_prev[1:]).sum() +d[0]   
        for i in range(1,n):
            x_n[i]= np.matmul(b[i,:i],x_n[:i]).sum() + np.matmul(b[i,i+1:],x_prev[i+1:]).sum() +d[i] 
            
    return k, x_n
    

>#### ``Seidel_solver`` -- функція, яка організовує обчислення наближеного розв'язку СЛАР (1) методом Зейделя з точністю ``eps`` 
-------------------

In [11]:
eps=0.001
k, xk = Seidel_solver(set_matrix, set_vector, set_x0, eps, max_iter)

if k > 0 and k < max_iter:
    print(f"Чисельний розв'язок системи \n x={xk} \n обчислено за {k} ітерацій")
    print(f"похибка чисельного розв'язку {norm_3(xk-x)}.")
elif  k == max_iter : 
    print(f"Чисельний розв'язок системи не обчислено за {k} ітерацій")
else:
    print("Матриця СЛАР (1) не задовольняє достатню умову збіжності ітерацій")

Чисельний розв'язок системи 
 x=[1.00008551 1.99994699 2.99998085] 
 обчислено за 5 ітерацій
похибка чисельного розв'язку 0.00010241602653679628.


>#### Функції для задання матриці та векторів вільних членів та початкового наближення конкретних СЛАР
*   ``set_matrix`` - функція, яка задає і повертає як результат матрицю СЛАР
*   ``set_vector`` - функція, яка задає і повертає як результат вектор вільних членів 
*   ``set_x0`` - функція, яка задає і повертає як результат вектор початкового наближення
*   виконати відповідну комірку з визначеннм цих функцій кожного разу, коли матрицю або вектор змінюють 

#### Обчислювальні експерименти
------------

Продемонструємо на прикладах застосування ітераційного методу Зейделя  до розв'язування СЛАР.

**Приклад 1.** (приклад 2.4) Обчислити методом Зейделя розв'язок СЛАР
\begin{equation}\label{2.2.2}
\left\{\begin{array}{rcl}
10 x_1 + \;\, x_2 +\;\; 2 x_3 & = 18 \\
\quad x_1 + 5 x_2 - \; \;\; x_3  & = 8 \\
\quad x_1 - 2 x_2 + 10 x_3  & = 27\,
\end{array} \right. 
\end{equation}

Легко переконатися, що вектор  $x=(1,2,3)^\top$ є точним розв'язком цієї системи, збережемо це значення:

In [6]:
x=np.array([1, 2, 3])

Підготуємо функції, які задаватимуть матрицю СЛАР та вектори вільних членів $d$ і початкового наближення $x^0$:

In [7]:
def set_matrix():
    """ функція для задання матриці СЛАР"""
    matrix=np.array([[10, 1, 2],[1,5,-1],[1,-2,10]],dtype=float )
    return matrix

In [8]:
def set_vector():
    """ функція для задання вектора вільних членів СЛАР"""
    vector=np.array([[18], [8], [27]],dtype=float)
    return vector

In [9]:
def set_x0():
    """ функція для задання вектора початкового наближення розв'язку"""
    return np.array([[1], [2], [1]],dtype=float)

Оскільки ми не робили оцінки необхідної кількості ітерацій, то наперед обмежимо їх достатньо великим числом: 

In [22]:
max_iter=100

Знайдемо чисельний розв'язок СЛАР, попередньо задавши значення параметра $eps$: 

In [11]:
eps=0.001
k, xk = Seidel_solver(set_matrix, set_vector, set_x0, eps, max_iter)

if k > 0 and k < max_iter:
    print(f"Чисельний розв'язок системи \n x={xk} \n обчислено за {k} ітерацій")
    print(f"похибка чисельного розв'язку {norm_3(xk-x)}.")
elif  k == max_iter : 
    print(f"Чисельний розв'язок системи не обчислено за {k} ітерацій")
else:
    print("Матриця СЛАР (1) не задовольняє достатню умову збіжності ітерацій")

Чисельний розв'язок системи 
 x=[1.00008551 1.99994699 2.99998085] 
 обчислено за 5 ітерацій
похибка чисельного розв'язку 0.00010241602653679628.


Можемо досягти більшої точності чисельного розв'язку, модифікуючи значення параметра $eps$: 

In [12]:
eps=0.00001
k, xk = Seidel_solver(set_matrix, set_vector, set_x0, eps, max_iter)

if k > 0 and k < max_iter:
    print(f"Чисельний розв'язок системи \n x={xk} \n обчислено за {k} ітерацій")
    print(f"похибка чисельного розв'язку {norm_3(xk-x)}.")
elif  k == max_iter : 
    print(f"Чисельний розв'язок системи не обчислено за {k} ітерацій")
else:
    print("Матриця СЛАР (1) не задовольняє достатню умову збіжності ітерацій")

Чисельний розв'язок системи 
 x=[1.00000097 1.9999994  2.99999978] 
 обчислено за 7 ітерацій
похибка чисельного розв'язку 1.1670628701770979e-06.


In [13]:
eps=0.0000001
k, xk = Seidel_solver(set_matrix, set_vector, set_x0, eps, max_iter)

if k > 0 and k < max_iter:
    print(f"Чисельний розв'язок системи \n x={xk} \n обчислено за {k} ітерацій")
    print(f"похибка чисельного розв'язку {norm_3(xk-x)}.")
elif  k == max_iter : 
    print(f"Чисельний розв'язок системи не обчислено за {k} ітерацій")
else:
    print("Матриця СЛАР (1) не задовольняє достатню умову збіжності ітерацій")

Чисельний розв'язок системи 
 x=[1. 2. 3.] 
 обчислено за 10 ітерацій
похибка чисельного розв'язку 1.4192169363730446e-09.


Якщо порівняти отримані чисельні розв'язки даної СЛАР з розв'язками методу Якобі, то можна побачити, що метод Зейделя тосягає потрібну точність за меншу кількість ітерацій.

Зейдель для матриці Гільберта

In [26]:
def Seidel_solver_Hilbert(n,set_matrix, set_vector,set_x0, eps, max_iter):
    """ Розв'язування СЛАР методом простих ітерацій """      
    b=set_matrix(n)  
    d=set_vector(n)
    Jacobi_modification(b, d) 
    k=0
    if norm_1(b)<10 or norm_2(b)<10 : 
        x0=set_x0(n)        
        k, x = Seidel_iteration(b,d,x0,eps,max_iter)
        return k, x   
    else:
        return k, d 
    

In [33]:
def set_matrix(n):
    """ функція для задання матриці конкретної СЛАР (по рядках)"""   
    a = np.empty((n,n),dtype=float)
    for i in range(n):
        for j in range(n):
            a[i,j] = 1/(i+j+1) 
    return a

In [34]:
def set_vector(n):
    """ функція для задання матриці конкретної СЛАР (по рядках)"""   
    b = np.zeros(n ,dtype=float)
    for i in range(n):
        for j in range(n):
            b[i] += 1/(i+j+1) 
    return b

In [38]:
def set_x0(n):
    """ функція для задання вектора початкового наближення розв'язку"""
    return set_vector(n)

In [24]:
n=5

In [32]:
eps=0.0000001
max_iter=100000
k, xk = Seidel_solver_Hilbert(n,set_matrix, set_vector, set_x0, eps, max_iter)

if k > 0 and k < max_iter:
    print(f"Чисельний розв'язок системи \n x={xk} \n обчислено за {k} ітерацій")
    #print(f"похибка чисельного розв'язку {norm_3(xk-x)}.")
elif  k == max_iter : 
    print(f"Чисельний розв'язок системи не обчислено за {k} ітерацій")
else:
    print("Матриця СЛАР (1) не задовольняє достатню умову збіжності ітерацій")

Чисельний розв'язок системи 
 x=[0.99998456 1.00028347 0.99879233 1.00180808 0.99912123] 
 обчислено за 60054 ітерацій


In [48]:
def Seidel_solver_HPS(n,set_matrix, set_vector,set_x0, eps, max_iter):
    """ Розв'язування СЛАР методом простих ітерацій """      
    aa=set_matrix(n)  
    bb=set_vector(n)
    aT=aa.T
    aTa=aT.dot(aa)
    aTb=aT.dot(bb)
    Jacobi_modification(aTa, aTb) 
    k=0
    if norm_1(aTa)<100 or norm_2(aTa)<100 : 
        x0=set_x0(n)        
        k, x = Seidel_iteration(aTa,aTb,x0,eps,max_iter)
        return k, x   
    else:
        return k, bb 
    

In [51]:
eps=0.00001
max_iter=100000
n=15
k, xk = Seidel_solver_HPS(n,set_matrix, set_vector, set_x0, eps, max_iter)

if k > 0 and k < max_iter:
    print(f"Чисельний розв'язок системи \n x={xk} \n обчислено за {k} ітерацій")
    #print(f"похибка чисельного розв'язку {norm_3(xk-x)}.")
elif  k == max_iter : 
    print(f"Чисельний розв'язок системи не обчислено за {k} ітерацій")
else:
    print("Матриця СЛАР (1) не задовольняє достатню умову збіжності ітерацій")

Чисельний розв'язок системи 
 x=[1.00142376 1.0333191  0.90862127 0.96381889 1.01723252 1.04820861
 1.05983009 1.05769717 1.04638801 1.02917617 1.00831087 0.98531729
 0.96122468 0.93672516 0.91228149] 
 обчислено за 17079 ітерацій
